In [2]:
# Importing modules used for file management
import os 
import sys

# Importing modules used for data analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# the next line is necessary to display plots in Jupyter
%matplotlib inline

# Create file path for battery md module
# Battery md from Eugene Ragasa and Vincent La
battery_md_path = "/Users/kims3/Documents/projects/2CEE_Project/repo/battery_md-main/src/"
sys.path.append(battery_md_path)

# Importing premade classes and methods to analyze NVT data
# Modules made by Eugene Ragasa and Vincent La
import MDAnalysis as mda
from MDAnalysis.analysis import rms, msd
from sklearn.linear_model import LinearRegression
from battery_md.jobs import Nvt
from battery_md.mixtures import ElectrolyteMixture
# from battery_md.io.lammps.diffusion import calculate_diffusion_coefficient

In [3]:
def check_if_project_path_exists(project_path):
    project_path_exists = os.path.isdir(project_path)
    print(f"project_path:{project_path}")
    print(f"project_path_exists:{project_path_exists}")
    return os.path.isdir(project_path_exists)

def get_mixture_name(mixture, timesteps):
    mixture_ratios = [v for v in mixture['solvent_composition_by_volume'].values()]
    str_mixture_ratios = ":".join([str(k) for k in mixture_ratios])
    mixture_name = "_".join([timesteps, str_mixture_ratios])
    return mixture_name

def get_sim_name(mixture, T, timesteps):
    mixture_ratios = [v for v in mixture['solvent_composition_by_volume'].values()]
    str_mixture_ratios = "_".join([str(k) for k in mixture_ratios])
    molec_names = [k for k in mixture['solvent_composition_by_volume'].keys()]
    str_molec_names = "_".join([str(k) for k in molec_names])
    temp = str(T) + "K"
    fileName = "_".join([timesteps, mixture.get('solute_symbol'), str_mixture_ratios, str_molec_names, "nvt", temp])
    return fileName
    

In [4]:
def calculate_rmsd(trajectory_path='system.lammpstrj',
                   topology_path='system.start.data',
                   atom_symbols=["Li", "P"]):
    
    FS2PS = 1/1000 # femtoseconds to picoseconds

    timestep_in_fs = 1
    timesteps_between_dumps = 100

    atom_style = "id resid type x y z ix iy iz"

    dt = timestep_in_fs * timesteps_between_dumps * FS2PS

    u = mda.Universe(
        topology_path,  
        trajectory_path,
        topology_format = 'DATA', 
        format = 'LAMMPSDUMP', 
        atom_style = atom_style,
        dt = dt
    )
    

    # should replace this with something a little more robust
    atomic_masses = {'Li':6.941, 'P':30.9738}
    
    rmsd = {}
    for s, amu in atomic_masses.items():
        atom_type = u.atoms.groupby('masses')[atomic_masses[s]][0].type
        rmsd[s] = rms.RMSD(u, select=f"type {atom_type}", dt=dt)
        rmsd[s].run()
        
    return rmsd

In [6]:
project_path = "000allMixtures"
project_path_exists = os.path.isdir(project_path)
print(f"project_path={project_path}")
print(f"project_path_exists={project_path_exists}")

# list of temps I held the simulation at
temperatures = [230, 240, 250, 260, 270, 280, 290, 300]

# temperature mixture
electrolyte_mixtures = [
    {'solute_n_moles': 1.0, 'solute_symbol': 'LiPF6', 'solvent_composition_by_volume': {'CEE': 100, 'MPN': 0}},
    {'solute_n_moles': 1.0, 'solute_symbol': 'LiPF6', 'solvent_composition_by_volume': {'CEE': 66, 'MPN': 33}}#,
#     {'solute_n_moles': 1.0, 'solute_symbol': 'LiPF6', 'solvent_composition_by_volume': {'CEE': 66, 'MPN': 33}},
#     {'solute_n_moles': 1.0, 'solute_symbol': 'LiPF6', 'solvent_composition_by_volume': {'CEE': 0, 'MPN': 100}}
]

atom_symbols = ["Li", "P"]
solvent_symbols = ['2CEE', '3MPN']


project_path=000allMixtures
project_path_exists=True


In [7]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [8]:
results = []

for idx_m, m in enumerate(electrolyte_mixtures):
    mixture_name = get_mixture_name(m, "100K")
    print(f"idx_m={idx_m}")
    print(f"m={m}")
    print(f"mixture_name={mixture_name}")
    
    for idx_T, T in enumerate(temperatures):
        print(f"idx_T={idx_T}")
        print(f"T={T}")
    
        result = {}
        result["mixture_name"] = mixture_name
        result["temperature"] = T
        
        job_name = os.path.join(project_path, get_sim_name(m, T, "100K"))
        trajectory_path = os.path.join(job_name, 'system.production.lammpstrj')
        topology_path = os.path.join(job_name, 'system.data')
        #configuration_path = os.path.join(job_name, 'configuration.yaml')
        ionic_concentration = 1.0
        degrees_of_freedom = 3
        
        # check if file exists
        print(f"\ntrajectory_path={trajectory_path}:{os.path.isfile(trajectory_path)}")
        print(trajectory_path, type(trajectory_path))
        
        print(f"\ntopology_path={topology_path}:{os.path.isfile(topology_path)}")
        print(topology_path, type(topology_path))
        
        print(f"\natom_symbols={atom_symbols}")
        print(atom_symbols, type(atom_symbols))

idx_m=0
m={'solute_n_moles': 1.0, 'solute_symbol': 'LiPF6', 'solvent_composition_by_volume': {'CEE': 100, 'MPN': 0}}
mixture_name=100K_100:0
idx_T=0
T=230

trajectory_path=000allMixtures/100K_LiPF6_100_0_CEE_MPN_nvt_230K/system.production.lammpstrj:True
000allMixtures/100K_LiPF6_100_0_CEE_MPN_nvt_230K/system.production.lammpstrj <class 'str'>

topology_path=000allMixtures/100K_LiPF6_100_0_CEE_MPN_nvt_230K/system.data:True
000allMixtures/100K_LiPF6_100_0_CEE_MPN_nvt_230K/system.data <class 'str'>

atom_symbols=['Li', 'P']
['Li', 'P'] <class 'list'>
idx_T=1
T=240

trajectory_path=000allMixtures/100K_LiPF6_100_0_CEE_MPN_nvt_240K/system.production.lammpstrj:True
000allMixtures/100K_LiPF6_100_0_CEE_MPN_nvt_240K/system.production.lammpstrj <class 'str'>

topology_path=000allMixtures/100K_LiPF6_100_0_CEE_MPN_nvt_240K/system.data:True
000allMixtures/100K_LiPF6_100_0_CEE_MPN_nvt_240K/system.data <class 'str'>

atom_symbols=['Li', 'P']
['Li', 'P'] <class 'list'>
idx_T=2
T=250

trajectory_path=00